In [12]:
from lora import generate, load_model
from models import ModelArgs, Model, LoRALinear
from mlx.utils import tree_map, tree_flatten, tree_unflatten

In [16]:
model, tokenizer = load_model("mistral-mlx")
model.load_weights("adapters.npz")

In [4]:
def load_and_generate(model_path, adapter_path, prompt, num_tokens=3):
    # Load the model and tokenizer
    model, tokenizer = load_model(model_path)

    # Apply LoRA adapters
    model.load_weights(adapter_path)

    # Generate tokens
    generated_tokens = []
    token_gen = generate(model, prompt, tokenizer, num_tokens)
    for token in token_gen:
        generated_tokens.append(token)

    return generated_tokens


In [5]:
from typing import NamedTuple
class Args(NamedTuple):
    prompt: str
    num_tokens: int
    temp: float

In [9]:
args = Args("User Profile: Alexander Roberts is a Investor in the Private Equity industry. Identifies and invests in high-growth potential businesses and startups. Alexander Roberts has a EducationLevels.MASTERS degree in Finance from New York City, New York, USA.. Article: From: tedebear@leland.Stanford.EDU (Theodore Chen)\nSubject: Re: Toyota wagons\nOrganization: DSG, Stanford University, CA 94305, USA\nLines: 15\n\nIn article <1993Apr5.213032.26844@pmafire.inel.gov> russ@pmafire.inel.gov (Russ Brown) writes:\n>In article <1pq6bl$9rj@news.ysu.edu> ak296@yfn.ysu.edu (John R. Daker) writes:\n>>Has anybody noticed that Toyota has an uncanny knack for designing horrible\n>>ugly station wagons?  Tercels, Corollas, Camrys.  Have their designers no\n>>aesthetic sense at all?\n>>-- \n>The new Camry Wagon may just be even uglier than my Tercel.   :-)\n\ntoyota has cornered the market on ugly station wagons.\nafter seeing the new camry sedan, i had thought toyota would\nfinally turn out something nice-looking.  the new camry station\nwagon bears a strong resemblance to a hearse, and a weird looking\none at that.\n\n-teddy\n. Relevant: ", 2, 0.8)

In [10]:
load_and_generate("mistral-mlx", "adapters.npz", "User Profile: Alexander Roberts is a Investor in the Private Equity industry. Identifies and invests in high-growth potential businesses and startups. Alexander Roberts has a EducationLevels.MASTERS degree in Finance from New York City, New York, USA.. Article: From: tedebear@leland.Stanford.EDU (Theodore Chen)\nSubject: Re: Toyota wagons\nOrganization: DSG, Stanford University, CA 94305, USA\nLines: 15\n\nIn article <1993Apr5.213032.26844@pmafire.inel.gov> russ@pmafire.inel.gov (Russ Brown) writes:\n>In article <1pq6bl$9rj@news.ysu.edu> ak296@yfn.ysu.edu (John R. Daker) writes:\n>>Has anybody noticed that Toyota has an uncanny knack for designing horrible\n>>ugly station wagons?  Tercels, Corollas, Camrys.  Have their designers no\n>>aesthetic sense at all?\n>>-- \n>The new Camry Wagon may just be even uglier than my Tercel.   :-)\n\ntoyota has cornered the market on ugly station wagons.\nafter seeing the new camry sedan, i had thought toyota would\nfinally turn out something nice-looking.  the new camry station\nwagon bears a strong resemblance to a hearse, and a weird looking\none at that.\n\n-teddy\n. Relevant: ", args)

User Profile: Alexander Roberts is a Investor in the Private Equity industry. Identifies and invests in high-growth potential businesses and startups. Alexander Roberts has a EducationLevels.MASTERS degree in Finance from New York City, New York, USA.. Article: From: tedebear@leland.Stanford.EDU (Theodore Chen)
Subject: Re: Toyota wagons
Organization: DSG, Stanford University, CA 94305, USA
Lines: 15

In article <1993Apr5.213032.26844@pmafire.inel.gov> russ@pmafire.inel.gov (Russ Brown) writes:
>In article <1pq6bl$9rj@news.ysu.edu> ak296@yfn.ysu.edu (John R. Daker) writes:
>>Has anybody noticed that Toyota has an uncanny knack for designing horrible
>>ugly station wagons?  Tercels, Corollas, Camrys.  Have their designers no
>>aesthetic sense at all?
>>-- 
>The new Camry Wagon may just be even uglier than my Tercel.   :-)

toyota has cornered the market on ugly station wagons.
after seeing the new camry sedan, i had thought toyota would
finally turn out something nice-looking.  the new 

['*', '*', '*', '*', '\n']

In [ ]:
model, tokenizer = load_model(args.model)

    # Freeze all layers other than LORA linears
model.freeze()
for l in model.layers[-args.lora_layers :]:
    l.attention.wq = LoRALinear.from_linear(l.attention.wq)
    l.attention.wv = LoRALinear.from_linear(l.attention.wv)

p = sum(v.size for _, v in tree_flatten(model.parameters())) / 10**6
print(f"Total parameters {p:.3f}M")
p = sum(v.size for _, v in tree_flatten(model.trainable_parameters())) / 10**6
print(f"Trainable parameters {p:.3f}M")

   # Load the LoRA adapter weights which we assume should exist by this point
model.load_weights("adapters.npz")
print("Generating")
generate(model, args.prompt, tokenizer, args)